In [1]:
# def get_code_data(path):
#     for _, _, files in os.walk(path):
#         data = np.empty((0, 100))
#         for file in tqdm_notebook(files):
#             with open(path + file, 'rb') as f:
#                 array = pickle.load(f)
#                 for i in tqdm_notebook(range(len(array)), leave=False):
#                     data = np.concatenate((data, array[i].inputs), axis=0)
#     return data


In [2]:
# data = get_code_data(base_path + '/after_preprocess/')

In [3]:
# import ast

In [4]:
# ast.dump(ast.parse('if x >= 0:\n    x = 0'))

In [5]:
# dir(ast.parse('test_var = test_list[0]').body[0])

## Импорты

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
from tqdm import tqdm_notebook, trange
import pickle
import numpy as np
from IPython.display import clear_output

# Предобработка

In [7]:
base_path = '/Users/dentarasov/Yandex.Disk.localized/current/vkr'
code_batched = base_path + '/after_preprocess/after_preprocess.part0'

In [8]:
def build_vocab():
    # load pre-computed vocab
    with open(base_path + '/mapping.map', 'rb') as f:
        word_to_id = pickle.load(f)
    id_to_word = dict([(v, k) for (k, v) in word_to_id.items()])
    return word_to_id, id_to_word

word_to_id, id_to_word = build_vocab()


In [9]:

params = {
    'batch_size': 64,
    'emb_size': 150,
    'vocab_size': len(word_to_id),
    # 'seq_len': 100,
}


In [10]:
# def get_code_data(path):
#     for _, _, files in os.walk(path):
#         data = np.empty((0, 100))
#         for file in tqdm_notebook(files):
#             with open(path + file, 'rb') as f:
#                 array = pickle.load(f)
#                 for i in tqdm_notebook(range(len(array)), leave=False):
#                     data = np.concatenate((data, array[i].inputs), axis=0)
#     return data


In [11]:
# data = get_code_data(base_path + '/after_preprocess/')

In [12]:
# data.shape

In [13]:
# data.astype('int64')

In [14]:
# with open('./after_preprocess/all_inputs', 'wb') as f:
#     pickle.dump(data.astype('int64'), f)

In [15]:
data_path = base_path + '/after_preprocess/all_inputs'

In [16]:
class CodeDataset(Dataset):
    def __init__(self, path, params):
        with open(path, 'rb') as f:
            self.data = pickle.load(f)
        print(type(self.data))
        print(type(self.data[0]))
#         self.batch_size = params['batch_size']

    def __len__(self):
#         # return number of batches
#         return int(np.ceil(self.data.shape[0] // self.batch_size))
        return self.data.shape[0]
    
    def __getitem__(self, index):
#         return self.data[index * self.batch_size : (index+1) * self.batch_size]
        return self.data[index]


In [17]:
train_loader = DataLoader(
    CodeDataset(data_path, params),
    batch_size=params['batch_size'],
    shuffle=True,
    num_workers=10
)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [18]:
# data[0:64].shape

In [19]:
# for i in train_loader:
#     print(type(i))
#     print(i.shape)
#     break

# Инициализация модели

In [25]:
!pip3 freeze

absl-py==0.7.0
appnope==0.1.0
asn1crypto==0.24.0
astor==0.7.1
atomicwrites==1.2.1
attrs==18.2.0
backcall==0.1.0
bleach==3.0.2
boto==2.49.0
boto3==1.9.67
botocore==1.12.67
bz2file==0.98
catboost==0.11.1
certifi==2018.11.29
cffi==1.12.2
chardet==3.0.4
cloudpickle==0.6.1
cryptography==2.6.1
cycler==0.10.0
Cython==0.29.6
dask==1.0.0
decorator==4.3.0
defusedxml==0.5.0
docutils==0.14
entrypoints==0.2.3
enum34==1.1.6
format-sql==0.12.0
funcy==1.11
future==0.17.1
gast==0.2.2
gensim==3.6.0
gitdb2==2.0.5
github3.py==1.3.0
GitPython==2.1.11
grpcio==1.18.0
h5py==2.9.0
idna==2.8
imageio==2.5.0
imageio-ffmpeg==0.2.0
ipykernel==5.1.0
ipython==7.1.1
ipython-genutils==0.2.0
ipywidgets==7.4.2
jedi==0.13.1
Jinja2==2.10
jmespath==0.9.3
joblib==0.13.0
jsonschema==2.6.0
jupyter==1.0.0
jupyter-client==5.2.3
jupyter-console==6.0.0
jupyter-core==4.4.0
jwcrypto==0.6.0
Keras==2.2.4
Keras-Applications==1.0.6
Keras-Preprocessing==1.0.5
kiwisolver==1.0.1
Markdown==3.0.1
MarkupSafe==1.1.0
matplotlib==3.0.2
mistune==

In [20]:
class LstmBaseline(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.params = params
        self.embedding = nn.Embedding(
            self.params['vocab_size'],
            self.params['emb_size']
        )
        self.lstm = nn.LSTMCell(
            self.params['emb_size'],
            self.params['emb_size'],
            2
        )
        self.linear = nn.Linear(
            self.params['emb_size'],
            self.params['vocab_size']
        )
    
    def init_hidden(self, batch_size):
        pass

    def forward(self, inputs, hidden):
        embs = self.embedding(inputs)
#         print(hidden.shape)
        print(embs.shape)
        print(embs.grad)
        # need 100, 64, 150(?)
        # real 64, 100, 150
        print('inputs.shape', inputs.shape)
        output, hidden = self.lstm.forward(embs, hidden)
        print(output.grad)
        output = self.linear(output)
        print(output.grad)
        return output, hidden


# Обучение

In [21]:
print(torch.from_numpy(np.arange(3)).grad)

None


In [22]:
def train_epoch(model, optimizer, lr):
    loss_log = []
    model.train()
    
    for batch_num, x in zip(trange(len(train_loader)), train_loader):
        optimizer.zero_grad()
        print(1)
        print('x', x.grad)
        print('x', type(x))
        print('x', x)
#         x = x.long()
#         x.requires_grad = True

#         x, y = x.to(device), y.to(device)
#         params['batch_size'] = x.shape[0]
#         p = torch.Tensor(generate_mask(params)).to(device)
        
        # todo 3rd parameter should be batch_size, shouldn't it?
#         hidden = torch.zeros(2, 2, x.shape[1], params["emb_size"]) #, device=device)

#         output, _ = model.forward(x)
        print(2)
        loss_value = 0.
#         loss = nn.CrossEntropyLoss()
        # todo check
        hidden = None
        for i in tqdm_notebook(range(x.shape[1] - 1)):
            output, hidden = model.forward(x[:, i], hidden)
            loss_value += F.cross_entropy(output[:, i], x[:, i + 1])
        print(3)
        print(type(loss_value))
        print(dir(loss_value))
        print(loss_value.grad)
        print(loss_value.requires_grad)
        print(loss_value.requires_grad_)
        loss_value.backward()
        print(3.5)
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)
        optimizer.step()
        print(4)
        
        loss_value = loss_value.item()
        loss_log.append(loss_value / x.shape[1])
    perp = np.exp(np.mean(loss_log))
    return loss_log, perp

def test(model, test_batches):
    loss_log = []
    model.eval()
    for batch_num, x in zip(trange(len(train_loader)), val_loader):        
        hidden = model.init_hidden(batch.shape[0])
        loss = 0
        output = model.forward(x, y)
        loss = F.cross_entropy(output.float(), y.float())
        loss = loss.item()
        loss_log.append(loss / nums.shape[0])
    return loss_log

def plot_history(train_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)
    plt.xlabel('train steps')
    plt.legend(loc='best')
    plt.grid()
    plt.show()
    
def train(model, opt, n_epochs):
    train_log = []
    val_log = []
    perp_log = []
    lr = 0.05
    lr_decay_base = 1 / 1.15
    m_flat_lr = 20.0
    for epoch in range(n_epochs):
        lr_decay = lr_decay_base ** max(epoch - m_flat_lr, 0)
        lr = lr * lr_decay
        train_loss, perp = train_epoch(model, opt, lr)
        train_log.extend(train_loss)
        perp_log.append(perp)
        if (epoch + 1) % 10 == 0:
            torch.save(
                {
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': opt.state_dict(),
                    'loss': train_log[-1]
                },
                './model_checkpoints/lstm_baseline_checkpoint_{}.pt'.format(epoch)
            )
        clear_output()
        print("Epoch:{}".format(epoch))
        plot_history(train_log)
        plot_history(perp_log)
    torch.save(
        {
            'epoch': n_epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': train_log[-1]
        },
        './model_checkpoints/lstm_baseline_checkpoint_{}.pt'.format(n_epochs)
    )
#     np.save("/home/.../model_checkpoints..._logs.npy", np.array(train_log))
#     np.save("/home/.../model_checkpoints..._logs_perp.npy", np.array(perp_log))
    
# model = LstmBaseline(params)

# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# train(model, optimizer, 10)


In [23]:
model = LstmBaseline(params)

In [24]:
%%time

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train(model, optimizer, 10)

  0%|          | 0/10871 [00:00<?, ?it/s]

1
x None
x <class 'torch.Tensor'>
x tensor([[    6, 25336,     6,  ...,     2,  4961,     7],
        [   22, 65627,     2,  ...,     0,     0,     0],
        [   23,    20,     4,  ...,    20,     4,     8],
        ...,
        [    4,     5,    28,  ...,    12,    17,  4853],
        [    6,   163,     4,  ...,  2339,     5,     2],
        [    2,    30,   124,  ...,    46,     6,  9691]])
2


torch.Size([64, 150])
None
inputs.shape torch.Size([64])



RuntimeError: input must have 3 dimensions, got 2